# Import libraries

In [2]:
import math
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
import scipy
from PIL import Image
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential 
from tensorflow.keras.metrics import Precision, Recall
import tensorflow.keras.layers as tfl
from tensorflow.python.framework import ops
from sklearn.preprocessing import LabelBinarizer

# Load the dataset

In [2]:
all_merged = pd.read_csv('C:/Users/s3929438/all_features_desktop_100_final_latest.csv')

# Select a single user
## Create the training and testing sets for the user


In [3]:
user_train_set = all_merged[all_merged['User']==1].sample(frac=0.7, random_state=42)
sample_ids = all_merged[all_merged['User']==1].sample(frac=0.7, random_state=42)['sample']
user_test_set = all_merged[(all_merged['User']==1) & ~all_merged['sample'].isin (sample_ids)]

## Create the training and testing sets for all other users

In [4]:
all_train_set  = pd.DataFrame()
all_test_set = pd.DataFrame()
user_id = 1
for i in range (1,117):
    if i != user_id:
        train_set = all_merged[all_merged['User']==i].sample(frac=0.7, random_state=42)
        sample_ids = all_merged[all_merged['User']==i].sample(frac=0.7, random_state=42)['sample']
        test_set = all_merged[(all_merged['User']==i) & ~all_merged['sample'].isin (sample_ids) ]
    
        all_train_set = all_train_set.append(train_set,ignore_index=True)
        all_test_set = all_test_set.append(test_set, ignore_index=True)

# Add a new column by labelling the User data as 1 and all others data as 0

In [5]:
user_train_set['User_type']=1
all_train_set['User_type']=0
user_test_set['User_type']=1
all_test_set['User_type']=0

C:\Users\s3929438\AppData\Local\Temp/ipykernel_10628/4033866476.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_test_set['User_type']=1


# Combine all users and build the final datasets

In [6]:
final_train_set = user_train_set.append(all_train_set)
final_test_set = user_test_set.append(all_test_set)

# Define the x_train, x_test, y_train and y_test

In [7]:
X_train = pd.DataFrame(final_train_set.iloc[:,3:92])
y_train =final_train_set.iloc[:,92]
X_test = pd.DataFrame(final_test_set.iloc[:,3:92])
y_test = final_test_set.iloc[:,92]

In [8]:
feature_names = X_train.columns

# Oversample the training set

In [9]:
sm = SMOTE()
x_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# Normalised x_train and x_test

In [10]:
scaled_data = MinMaxScaler()
scaled_train = pd.DataFrame(scaled_data.fit_transform(x_train_res), columns=feature_names)
scaled_test = pd.DataFrame(scaled_data.transform(X_test), columns=feature_names)

In [ ]:
# def KeystrokeModel():

#     model = tf.keras.Sequential([
           
#             tfl.ZeroPadding2D(padding=(0,0) , input_shape = (8850, 89,1), data_format="channels_last"),
#             tfl.Conv2D(32, (100,89), strides = (90,1)),
#             tfl.BatchNormalization(axis= 3),
#             tfl.ReLU(max_value=None, negative_slope=0.0, threshold=0.0),
# #             tfl.MaxPool2D(pool_size=(5, 89),strides=None,data_format=None),
# #             tfl.Conv2D(16, (5,5), strides = (4,4)),
# #             tfl.BatchNormalization(axis= 3),
# #             tfl.ReLU(max_value=None, negative_slope=0.0, threshold=0.0),
# #             tfl.MaxPool2D(pool_size=(2, 2),strides=None,data_format=None),
#             tfl.Flatten(),
#             tfl.Dense(3136,activation='relu'),
#             tfl.Dense(1,activation='sigmoid')
#         ])
    
#     return model

In [ ]:
# model = KeystrokeModel()
# model.compile(optimizer='adam',
#                    loss='binary_crossentropy',
#                    metrics=['accuracy'])

# Apply general deep learning model

### This model working fine. But the results we achived are very low. 

In [15]:
X = scaled_train.values
Y = (pd.DataFrame(y_train_res).values)
model = Sequential()
model.add(tfl.Dense(16, input_shape=(X.shape[1],), activation='relu')) # Add an input shape! (features,)
model.add(tfl.Dense(32, activation='relu'))
model.add(tfl.Dense(1, activation='sigmoid'))
model.summary() 
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy', Precision() ,Recall()])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 16)                1440      
                                                                 
 dense_6 (Dense)             (None, 32)                544       
                                                                 
 dense_7 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,017
Trainable params: 2,017
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(X,Y,epochs=80, 
                    batch_size=10,
                    shuffle=True,
                    verbose=1)

Epoch 1/80
885/885 [==============================] - 6s 4ms/step - loss: 0.2528 - accuracy: 0.9208 - precision_1: 0.8985 - recall_1: 0.9487
Epoch 2/80
885/885 [==============================] - 3s 4ms/step - loss: 0.1012 - accuracy: 0.9702 - precision_1: 0.9441 - recall_1: 0.9995
Epoch 3/80
885/885 [==============================] - 3s 4ms/step - loss: 0.0752 - accuracy: 0.9771 - precision_1: 0.9561 - recall_1: 1.0000
Epoch 4/80
885/885 [==============================] - 3s 4ms/step - loss: 0.0561 - accuracy: 0.9841 - precision_1: 0.9699 - recall_1: 0.9991
Epoch 5/80
885/885 [==============================] - 3s 4ms/step - loss: 0.0434 - accuracy: 0.9879 - precision_1: 0.9764 - recall_1: 1.0000
Epoch 6/80
885/885 [==============================] - 4s 4ms/step - loss: 0.0355 - accuracy: 0.9905 - precision_1: 0.9820 - recall_1: 0.9993
Epoch 7/80
885/885 [==============================] - 3s 4ms/step - loss: 0.0291 - accuracy: 0.9919 - precision_1: 0.9842 - recall_1: 0.9998
Epoch 8/80
88

In [17]:
model.evaluate(scaled_test, y_test)

60/60 [==============================] - 1s 4ms/step - loss: 0.0816 - accuracy: 0.9891 - precision_1: 0.2857 - recall_1: 0.2667


[0.08161916583776474,
 0.9890568256378174,
 0.2857142984867096,
 0.2666666805744171]

# Applying CNN model 

In [1]:
# The x_train and y_train are reshape to input to the CNN model

X_tr = (scaled_train.values).reshape(8850,89,1,-1)
Y_tr = (pd.DataFrame(y_train_res).values).reshape(8850,1,-1)

NameError: name 'scaled_train' is not defined

In [12]:
model = Sequential()
model.add(tfl.Conv2D(32, input_shape = (8850, 89,1), kernel_size=(50,89)))
# model.add(tfl.Conv2D(64, kernel_size=(5,5), activation='relu'))
model.add(tfl.Flatten())
model.add((tfl.Dense(64, activation='relu')))
model.add(tfl.Dense(1, activation='sigmoid'))
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 8801, 1, 32)       142432    
                                                                 
 flatten (Flatten)           (None, 281632)            0         
                                                                 
 dense_3 (Dense)             (None, 64)                18024512  
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 18,167,009
Trainable params: 18,167,009
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.fit(X_tr,Y_tr,epochs=80)

Epoch 1/80


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_2/dense_5/Relu' defined at (most recent call last):
    File "C:\Users\s3929438\Anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\s3929438\Anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\s3929438\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\s3929438\Anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\s3929438\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\s3929438\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\s3929438\Anaconda3\lib\asyncio\base_events.py", line 596, in run_forever
 

# Applying LSTM model

In [ ]:
model = Sequential()
model.add(LSTM(64, input_shape=(8850,89), activation='relu'))
model.add(Dropout(0.5))
# model.add(LSTM(128))
# model.add(Dropout(0.5))
# model.add(LSTM(256))
# model.add(Dropout(0.5))
# model.add(LSTM(64))
# model.add(Dropout(0.5))
model.add(Dense(8850, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', keras.metrics.Precision(),
                                                                     keras.metrics.Recall()])

In [ ]:
X = np.array(scaled_train).reshape((-1,8850,89))
Y = np.array(y_train_res).reshape((-1,8850))

In [ ]:
X_tst = np.array(scaled_test).reshape((-1,1919,89))
Y_tst = np.array(y_test).reshape((-1,1919))

In [ ]:
scaled_test.shape

In [ ]:
X

In [ ]:
model.fit(X,Y, epochs=50, shuffle=True, verbose=2)

In [ ]:
Performance = model.evaluate(X_tst, Y_tst, verbose=0)

In [ ]:
Performance